In [ ]:
# Importing necessary libraries
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Proposed Hybrid model 

These are all the output csvs we got from our four models ANN, SARIMA , LOGISTIC REGRESSION AND ANN CLASSFICATION MODELS. These csvs are having pelds forecasted . these csvs are used as inputs for hybrid model. The content in this csvs are pelds forecasted by individual models throughout 2021.We are training our hybrid model with these csvs and comparing them with actual peak flags of 2021.

These are all the csvs we are using as inputs :

sarima_df = pd.read_csv('sarima_rolling_forecast5_flagged_daily999.csv')
ann_forecast_df = pd.read_csv('actual_vs_forecast_with_datetimeANN_flagged_daily999.csv')
logistic_regression_df = pd.read_csv('Logistic_Regression_PLED_Classification_Daily999.csv')
ann_classification_df = pd.read_csv('Artificial_Neural_Network_PLED_Classification_Daily999.csv')

In [1]:
# Loading the output CSV files from all the models
sarima_df = pd.read_csv('sarima_rolling_forecast5_flagged_daily999.csv')
ann_forecast_df = pd.read_csv('actual_vs_forecast_with_datetimeANN_flagged_daily999.csv')
logistic_regression_df = pd.read_csv('Logistic_Regression_PLED_Classification_Daily999.csv')
ann_classification_df = pd.read_csv('Artificial_Neural_Network_PLED_Classification_Daily999.csv')

# Merging all dataframes on the Date column to prepare for the hybrid model
combined_df = sarima_df[['Date', 'Actual_Peak_Flag']].copy()
combined_df = combined_df.merge(sarima_df[['Date', 'Forecast_Peak_Flag']], on='Date', suffixes=('', '_SARIMA'))
combined_df = combined_df.merge(ann_forecast_df[['Date', 'Forecast_Peak_Flag']], on='Date', suffixes=('', '_ANN_Forecast'))
combined_df = combined_df.merge(logistic_regression_df[['Date', 'PLED_Prediction']], on='Date', suffixes=('', '_Logistic'))
combined_df = combined_df.merge(ann_classification_df[['Date', 'PLED_Prediction']], on='Date', suffixes=('', '_ANN_Classification'))

# Renaming columns for clarity
combined_df.rename(columns={
    'Forecast_Peak_Flag': 'SARIMA_Prediction',
    'Forecast_Peak_Flag_ANN_Forecast': 'ANN_Forecast_Prediction',
    'PLED_Prediction': 'Logistic_Prediction',
    'PLED_Prediction_ANN_Classification': 'ANN_Classification_Prediction'
}, inplace=True)

# Helper function to create confusion matrix as a dictionary
def calculate_confusion_matrix(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return {
        'True Positive (TP)': tp,
        'True Negative (TN)': tn,
        'False Positive (FP)': fp,
        'False Negative (FN)': fn
    }

# Helper function to calculate performance metrics
def calculate_metrics(conf_matrix):
    tp = conf_matrix['True Positive (TP)']
    tn = conf_matrix['True Negative (TN)']
    fp = conf_matrix['False Positive (FP)']
    fn = conf_matrix['False Negative (FN)']
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    inaccuracy = (fp + fn) / (tp + tn + fp + fn)
    
    return {
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Inaccuracy': inaccuracy
    }

# Calculating confusion matrices and metrics for individual models
sarima_results = calculate_confusion_matrix(combined_df['Actual_Peak_Flag'], combined_df['SARIMA_Prediction'])
sarima_metrics = calculate_metrics(sarima_results)

ann_forecast_results = calculate_confusion_matrix(combined_df['Actual_Peak_Flag'], combined_df['ANN_Forecast_Prediction'])
ann_forecast_metrics = calculate_metrics(ann_forecast_results)

logistic_results = calculate_confusion_matrix(combined_df['Actual_Peak_Flag'], combined_df['Logistic_Prediction'])
logistic_metrics = calculate_metrics(logistic_results)

ann_classification_results = calculate_confusion_matrix(combined_df['Actual_Peak_Flag'], combined_df['ANN_Classification_Prediction'])
ann_classification_metrics = calculate_metrics(ann_classification_results)

# Creating a dataframe to display the confusion matrices and metrics for individual models
data = {
    'Model': ['SARIMA', 'ANN Forecast', 'Logistic Regression', 'ANN Classification'],
    'True Positive (TP)': [sarima_results['True Positive (TP)'], ann_forecast_results['True Positive (TP)'],
                           logistic_results['True Positive (TP)'], ann_classification_results['True Positive (TP)']],
    'True Negative (TN)': [sarima_results['True Negative (TN)'], ann_forecast_results['True Negative (TN)'],
                           logistic_results['True Negative (TN)'], ann_classification_results['True Negative (TN)']],
    'False Positive (FP)': [sarima_results['False Positive (FP)'], ann_forecast_results['False Positive (FP)'],
                            logistic_results['False Positive (FP)'], ann_classification_results['False Positive (FP)']],
    'False Negative (FN)': [sarima_results['False Negative (FN)'], ann_forecast_results['False Negative (FN)'],
                            logistic_results['False Negative (FN)'], ann_classification_results['False Negative (FN)']],
    'Accuracy (%)': [sarima_metrics['Accuracy'] * 100, ann_forecast_metrics['Accuracy'] * 100,
                     logistic_metrics['Accuracy'] * 100, ann_classification_metrics['Accuracy'] * 100],
    'Sensitivity (%)': [sarima_metrics['Sensitivity'] * 100, ann_forecast_metrics['Sensitivity'] * 100,
                        logistic_metrics['Sensitivity'] * 100, ann_classification_metrics['Sensitivity'] * 100],
    'Inaccuracy (%)': [sarima_metrics['Inaccuracy'] * 100, ann_forecast_metrics['Inaccuracy'] * 100,
                       logistic_metrics['Inaccuracy'] * 100, ann_classification_metrics['Inaccuracy'] * 100]
}

# Meta-Learning using RandomForestClassifier
# Prepare the features for the meta-classifier
X_meta = combined_df[['SARIMA_Prediction', 'ANN_Forecast_Prediction', 'Logistic_Prediction', 'ANN_Classification_Prediction']]
y_meta = combined_df['Actual_Peak_Flag']

# Training a RandomForestClassifier as the meta-classifier
meta_clf = RandomForestClassifier(n_estimators=100, random_state=42)
meta_clf.fit(X_meta, y_meta)

# Making predictions using the meta-classifier
combined_df['Meta_Classifier_Prediction'] = meta_clf.predict(X_meta)

# Calculating confusion matrix and metrics for meta-classifier hybrid model
meta_classifier_results = calculate_confusion_matrix(combined_df['Actual_Peak_Flag'], combined_df['Meta_Classifier_Prediction'])
meta_classifier_metrics = calculate_metrics(meta_classifier_results)

# Addding Meta Classifier results to the dataframe
data['Model'].append('Meta Classifier Hybrid')
data['True Positive (TP)'].append(meta_classifier_results['True Positive (TP)'])
data['True Negative (TN)'].append(meta_classifier_results['True Negative (TN)'])
data['False Positive (FP)'].append(meta_classifier_results['False Positive (FP)'])
data['False Negative (FN)'].append(meta_classifier_results['False Negative (FN)'])
data['Accuracy (%)'].append(meta_classifier_metrics['Accuracy'] * 100)
data['Sensitivity (%)'].append(meta_classifier_metrics['Sensitivity'] * 100)
data['Inaccuracy (%)'].append(meta_classifier_metrics['Inaccuracy'] * 100)

# Displaying the results in a table format
results_df = pd.DataFrame(data)
print(results_df)

# Calculating the number of actual peaks, hybrid model peaks, and achieved peaks for Meta Classifier Hybrid
meta_classifier_peaks = combined_df['Meta_Classifier_Prediction'].sum()
achieved_peaks_meta = ((combined_df['Meta_Classifier_Prediction'] == 1) & (combined_df['Actual_Peak_Flag'] == 1)).sum()

# Output the results for Meta Classifier Hybrid model
results = {
    'Meta Classifier Hybrid': {
        'Number of Hybrid Model Peaks': meta_classifier_peaks,
        'Number of Achieved Peaks': achieved_peaks_meta
    }
}

# Displaying results for Meta Classifier Hybrid model
print("\nResults for Meta Classifier Hybrid Model:")
print(pd.DataFrame(results))

# Saving actual and hybrid peak flags in a CSV with the date column
output_df = combined_df[['Date', 'Actual_Peak_Flag', 'Meta_Classifier_Prediction']]
output_df.rename(columns={'Meta_Classifier_Prediction': 'Hybrid_Peak_Flag'}, inplace=True)
output_df.to_csv('actual_vs_hybrid_peak_flags.csv', index=False)

# Printing message indicating that the file has been saved
print("\nActual and Hybrid peak flags saved to 'actual_vs_hybrid_peak_flags.csv'")


                    Model  True Positive (TP)  True Negative (TN)  \
0                  SARIMA                  48                 284   
1            ANN Forecast                  53                 282   
2     Logistic Regression                  64                 222   
3      ANN Classification                  52                 286   
4  Meta Classifier Hybrid                  59                 283   

   False Positive (FP)  False Negative (FN)  Accuracy (%)  Sensitivity (%)  \
0                   13                   20     90.958904        70.588235   
1                   15                   15     91.780822        77.941176   
2                   75                    4     78.356164        94.117647   
3                   11                   16     92.602740        76.470588   
4                   14                    9     93.698630        86.764706   

   Inaccuracy (%)  
0        9.041096  
1        8.219178  
2       21.643836  
3        7.397260  
4        6.30137

/var/folders/tn/v941jp4540374j7s4hp2b37w0000gn/T/ipykernel_43550/365307018.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df.rename(columns={'Meta_Classifier_Prediction': 'Hybrid_Peak_Flag'}, inplace=True)


#### RESULT :
    Results for Meta Classifier Hybrid Model:
                              Meta Classifier Hybrid
               Number of Achieved Peaks                          59
               Number of Hybrid Model Peaks                      73